In [3]:
import numpy as np
import random

### Functions for Mahjong Rules

In [14]:
def initialize_game_state():
    state = np.zeros(3*9+7)
    return state

class Tile:
    def __init__(self, tileno, suit, copyno):
        self.tileno = tileno
        self.suit = suit
        self.copyno = copyno # To ensure that each tile is unique, assign a copy number to the tile (just in case)

class Draw_Pile(object):
    def __init__(self):
        self.deck = []
        for suit in ['Tong','Wan','Suo']:
            for tileno in range(1,10):
                for copyno in range(1,5):
                    tile = Tile(tileno, suit, copyno)
                    self.deck.append(tile)
        # Assign number 0 for the wind and dragon tiles as placeholder
        for suit in ['Dong','Nan','Xi','Bei','Bai_Ban','Fa_Cai','Hong_Zhong']:
            for copyno in range(1,5):
                tile = Tile(0, suit, copyno)
                self.deck.append(tile)
    
    def __len__ (self):
        return len(self.deck)
    
    # Deal a tile from index 0 to the player and removes it from Draw_Pile.deck
    def deal(self):
        if len(self.deck) == 0:
            return None
        else:
            return self.deck.pop(0)

class Discard_Pile(object):
    def __init__(self):
        self.deck = []        
        
class Player(object):
    def __init__(self):
        self.hand = []
        # The fourth piece of a kong is not considered one of the 13 tiles a player must always have in their hand.
        # For every melded kong, hand size increases by 1, so we need to track this
        self.melded_kong_count = 0
        self.melded_hand = []
        
    def discard(self,tile):
        if len(self.hand) == 0:
            return None
        else:
            return self.hand.pop(tile)

def score_hand(hand, test_mode = 0):
    #### Mahjong official rules scoring function to go here
    sum_of_hand = np.sum(hand, axis = 1)%10
    # Find singular tiles
    for start, stop in zip([0,9,18], [9,18,27])
        sum_of_hand_slice = sum_of_hand[start:stop]
        singular_tileno = argwhere(sum_of_hand_slice == 1)
        for i in singular_tileno:
            if i == 0 and (sum_of_hand_slice[1] == 0 or sum_of_hand_slice[2] == 0:
                if test: print('Singluar tiles at 1')
                return 0
            elif i == 8 and (sum_of_hand_slice[-2] == 0 or sum_of_hand_slice[-3] == 0):
                if test: print('Singluar tiles at 9')
                return 0
            elif sum_of_hand_slice[i-1] == 0 and sum_of_hand_slice[i+1] == 0 or  
                
    
    
    if any(copy_of_hands > 4):
        print(hand)
        raise Exception("score_hand funciton error: copy_of_hand out of range")
    # Thirteen Orphans
    if np.bincount(hand[0,:])[1] == 1 and np.bincount(hand[8,:])[1] == 1 and np.bincount(hand[9,:])[1] == 1 and np.bincount(hand[17,:])[1] == 1 and np.bincount(hand[18,:])[1] == 1 and np.bincount(hand[26,:])[1] == 1:
        if any(np.sum(hand[27:,:],axis = 1)%10 == 3):
            break
        elif np.bincount(np.sum(hand[25:,:],axis = 1)%10)[1] == 6:
            if test_mode: print('Wining Hand: Thirteen Orphans (13)')
            return 13
        
    # Great Winds
    if all(np.sum(hand[27:31,:], axis = 1)%10 == 3*np.ones([4])) and any(np.sum(hand, axis = 1)%10 == 2):
        if test_mode: print('Wining Hand:Great Winds (13)')
        return 13
    
    # All Honor Tiles
    if np.sum(hand[0:27], axis = 1)%10 == np.zeros([27]) and np.bincount(np.sum(hand[27:], axis = 1))[3] == 4 and np.bincount(np.sum(hand[27:], axis = 1))[2] == 1:
        if np.bincount(np.sum(hand[27:], axis = 1))[3] >= 0:
            extra_point = np.bincount(np.sum(hand[27:], axis = 1))[3]
        else:
            extra_point = 0
        
        if test_mode:print('Wining Hand: All Honor Tiles {}'.format(10+extra_point))
        return 10+extra_point
    
    # Nine Gates
    if sum(hand[9,:])%10 >= 3 and sum(hand[17,:])%10 >= 3 and np.sum(hand[10:17,:], axis = 1)%10 >=1:
        if test_mode:print('wining Hand: Nine Gates (10)')
        return 10
    
    # Orphans
    if np.bincount(sum_of_hand[[0,8,9,17,18,26],0])[3] == 4 and np.bincount(sum_of_hand[[0,8,9,17,18,26],0])[2] == 3:
        if test_mode:print('Wining Hand: Orpahns')
        return 10
        
    # Great Dragons (Need To DOUBLE CHECK)
    if sum_of_hand[31:,0] == 3*np.ones([3,1]) and any(sum_of_hand[0:31:,0] >= 2):
        if any(sum_of_hand[0:31:,0] == 3) and any(sum_of_hand[0:31:,0] == 2):
            if test_mode:print('Wining Hand: Great Dragons (Triplets and eyes)')
            return 8
        
        elif any(sum_of_hand[0:31:,0] == 2) and not any(sum_of_hand[0:31:,0] == 3):
            sum_of_hand[np.argwhere(sum_of_hand[0:31:,0] == 2)] -= 2
            if all(np.diff(np.nonzero(sum_of_hand[0:31:,0]), axis = 0) == 1):
                if test_mode:print('Wining Hand: Great Dragons (consecutive and eyes)')
                return 8
            else:
                break
                
        elif any(sum_of_hand[0:31:,0] == 3) and not any(sum_of_hand[0:31:,0] == 2):  
            sum_of_hand[np.argwhere(sum_of_hand[0:31:,0] == 3)] -= 2
            if all(np.diff(np.nonzero(sum_of_hand[0:31:,0]), axis = 0) == 1):
                if test_mode:print('Wining Hand: Great Dragons (consecutive and eyes)')
                return 8
            else:
                break
        else:
            break
    # All One Suit
    if sum_of_hand[9:] == 0:
        copy_of_hands =  sum_of_hand[0:9]
        count_tracking = copy_of_hands.copy()
        all_one_suit = 1
    elif (sum_of_hand[0:9] == 0 and sum_of_hand[18:] == 0):
        copy_of_hands =  sum_of_hand[9:18]
        all_one_suit = 1
    elif (sum_of_hand[0:18] == 0 and sum_of_hand[27:] == 0):
        copy_of_hands =  sum_of_hand[18:27]
        all_one_suit = 1
        
    # Meld-eye Count    
    chow_count = 0
    pong_count = 0
    eye_count = 0
    count_tracking = copy_of_hands[0,27].copy()
    for i, copy_of_hands in enumerate(cpoy_of_hands[0,27]):
        if count_tracking[i] == 0:
            pass
        else:
            if copy_of_hands == 3:
                if cpopy_of_hands[i+1:i+4] == [1,1,0]:
                    eye_count = 1
                    chow_count += 1
                    count_tracking[i] = 0
                    count_tracking[i+1:i+3] = [0,0]
                else:
                    pong_count += 1
                    count_tracking[i] == 0
            elif copy_of_hands == 4:
                pong_count += 1
                if cpoy_of_hand[i + 1] == 0 or cpoy_of_hand[i + 2] == 0:
                    if test_mode:print('Singluar Tiles (for test)')
                    break
                elif cpoy_of_hand[i + 1] > 0 and cpoy_of_hand[i + 2] > 0:
                    chow_count += 1
                    count_tracking[i+1:i+3] -= 1       
            elif copy_of_hands == 2:
                if eye_count == 0:
                    eye_count = 1
                    count_tracking[i] = 0
                elif eye_count == 1 and count_tracking[i+1:i+3] == [2,2]:
                    chow_count += 2
                    count_tracking[i:i+3] -= 2
                else:
                    break
            elif copy_of_hands == 1:
                if all(count_tracking[i+1:i+3] >= 1):
                    chow_count += 1
                    count_tracking[i:i+3] -= 1
    wind_count = np.bincount(cpoy_of_hands[27,31])
    wind_pong_count = wind_count[3]
    wind_eye_count = wind_count[2]
    dragon_count = = np.bincount(cpoy_of_hands[31:])
    dragon_pong_count = dragon_count[3]
    dragon_eye_count = dragon_count[2]
    
    if wing_pong_count == 2 and wind_eye_count == 1:
        if test: print('Small Wind')
        score += 6
    if dragon_pong_count == 2 and dragon_eye_count == 1:
        if test: print('Small Dragons')
        score += 5        
                    
    if all_one_suit:
        if test_mode: print('All in One Suit (testing)')
        score += 7
            
    if pong_count == 4 and eye_count == 1:
        if test_mode: print('All in Triplets (testing)')
        score += 3
    
        
            
            
                        
                    
                
                
        
        
        
        
        single_tiles = np.argwhere(sum_of_hand[0:31:,0] == 1)[:,0]
        for i in single_tiles:
            if sum_of_hand[i-1,0] == 0 and sum_of_hand[i+1,0] == 0: # Check if singular tiles has possible consecutive tiles
                break
            if i+1 in single_tiles and i+2 in single tiles:
                
                
        pairs_flag = 0
        
        if any(sum_of_hand[0:31:,0] == 2):
            sum_of_hand[np.argwhere(sum_of_hand[0:31:,0] == 2)] -= 2
            
            
    
            for n, i in enumerate(index):
                if i + 1 == index(n+1) and i + 2 == index(n+2):
                    
            
            
            
        
        
    
        
            
    
    
    return score

### Mahjong State Dictionary

In [5]:
mahjong_dict = {0:'1 Tong',1:'2 Tong',2:'3 Tong',3:'4 Tong',4:'5 Tong',5:'6 Tong',6:'7 Tong',7:'8 Tong',8:'9 Tong',
              9:'1 Wan',10:'2 Wan',11:'3 Wan',12:'4 Wan',13:'5 Wan',14:'6 Wan',15:'7 Wan',16:'8 Wan',17:'9 Wan',
              18:'1 Suo',19:'2 Suo',20:'3 Suo',21:'4 Suo',22:'5 Suo',23:'6 Suo',24:'7 Suo',25:'8 Suo',26:'9 Suo',
              27:'Dong',28:'Nan',29:'Xi',30:'Bei',
              31:'Bai_Ban',32:'Fa_Cai',33:'Hong_Zhong'}

In [42]:
random_seed = random.choices(np.arange(33*4),k=14)
state = np.zeros([33,4])
for i in random_seed:
    # int(i/4) gives tile no., i%4 gives copy no.
    state[int(i/4),i%4]=1
state

array([[1., 1., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 0.],
       [0., 0., 0., 1.]])

### Functions to View State

In [ ]:
# Use this function to output the tiles to see 
def display_tiles(pile):
    print('tile count =',len(pile))
    for tile in pile:
        print(tile.tileno, tile.suit)

### Functions for Decision Making

In [8]:
def decide(player_hand, drawn_tile, discard_pile):
    #### The decision function to eventually go here
    if winning_hand == 1:
        return score
    elif kong == 1:
        kong_count = kong_count + 1
        return kong_count
    else:
        return tile_to_discard
    
def value_function():
    #### If our value function is to differ from the official scoring rules
    value = score_hand()
    return value

In [4]:
a = np.matrix([[1],[1],[3],[2],[2],[4]])


In [5]:
np.argwhere(a == 1)[1,0]

1

In [6]:
a[2]

matrix([[3]])

In [15]:
all(0 in a, 5 in a)

TypeError: all() takes exactly one argument (2 given)

In [8]:
kk = []
for i in kk:
    print('test')

In [14]:
for j,k in zip([0,9,18], [9,18,27]):
    print(k)

9
18
27


TypeError: 'zip' object is not subscriptable